# Data Preprocessing

In [54]:
# from google.colab import drive
# drive.mount('/content/drive')

In [55]:
# %cd /content/drive/MyDrive/MSC_Intro_to_NLP_Group_Project/

In [56]:
!pip install nltk
!pip install gensim
!pip install 'transformers[torch]'
!pip install datasets
!pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [57]:
import json
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import re
import nltk
import string
from nltk import word_tokenize, pos_tag
import contractions
import pandas as pd
import spacy
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
!python -m spacy download en_core_web_sm

[nltk_data] Downloading package punkt to
[nltk_data]     /home/stud/giris1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/stud/giris1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 12.8 MB 4.4 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [58]:
# Using the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
gpu_name = torch.cuda.get_device_name(device)
print(gpu_name)

cuda
NVIDIA GeForce GTX 1080 Ti


In [59]:
# Load spaCy model outside of the function to avoid reloading it each time the function is called
nlp = spacy.load("en_core_web_sm")

In [60]:
# Function to capitalize the first letter of each sentence and proper nouns
def capitalize_first_and_proper_nouns(text):
    # Process the text using spaCy to create a Doc object
    doc = nlp(text)

    result = []

    # Iterate over the sentences in the Doc
    for sent in doc.sents:
        # Iterate over the tokens in the sentence
        for token in sent:
            # Capitalize the first letter of each sentence and proper nouns
            if token.is_sent_start or token.pos_ == 'PROPN':
                result.append(token.text.capitalize())
            else:
                result.append(token.text)

    # Rejoin the tokens into a single string
    return ' '.join(result)

# Function to remove repeated punctuations
def remove_repeated_punctuations(sentence):
    # Use regular expression to remove consecutive repeated punctuations
    cleaned_sentence = re.sub(r'(\W)\1+', r'\1', sentence)
    return cleaned_sentence

# Function to expand contractions
def expand_contractions(text):
    return contractions.fix(text)

# Define a tokenization function
def tokenize_sentences(sentences):
    return [word_tokenize(sentence) for sentence in sentences]

def fix_general_spacing(sentence):
    # Fix space before punctuation (like ' ,' to ',')
    sentence = re.sub(r'\s([,.?!:;])', r'\1', sentence)
    # Fix space after punctuation (like ' . ' to '. ')
    sentence = re.sub(r'([,.?!:;])\s', r'\1 ', sentence)
    # Fix space in contractions (like "don 't" to "don't")
    sentence = re.sub(r"\b(\w+)\s('t|'s|'m|'ll|'ve|'re|'d|n't)\b", r"\1\2", sentence)
    # Reduce multiple spaces between words to a single space
    sentence = re.sub(r'\s{2,}', ' ', sentence)
    return sentence

def preprocess(text):
    # text = text.lower()
    text = expand_contractions(text)
    text = remove_repeated_punctuations(text)
    text = capitalize_first_and_proper_nouns(text)
    text = fix_general_spacing(text)
    return text


In [61]:
# # Load the preprocessed data from the JSON file
# data_files={
#     "train":"data_train.json",
# }

# dataset = load_dataset("json", data_files=data_files)
# print(dataset)

json_file_path = "data_train.json"

with open(json_file_path, 'r') as file:
    dataset = json.load(file)


# a dataset class to load and preprocess the data

In [62]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "id": item["id"],
            "topic": item["topic"],
            "input_seq": torch.tensor(item["input_seq"]),  # Make sure this key matches the one used in numericalize_data
            "output_seq": torch.tensor(item["output_seq"]),  # Make sure this key matches the one used in numericalize_data
        }


Creating a Bi-Directional RNN Model

In [63]:
def preprocess_data(data):
    for item in data:
        item["transformation"]["informal"] = preprocess(item["transformation"]["informal"])
    return data

In [64]:
# preprocessed_data = preprocess_data(dataset)


In [65]:
def tokenize_data(data):
    for item in data:
        item["tokenized_informal"] = word_tokenize(item["transformation"]["informal"])
        item["tokenized_formal"] = word_tokenize(item["transformation"]["formal.ref0"])
    return data


In [66]:
def build_vocab(data):
    input_vocab = set()
    output_vocab = set()

    for item in data:
        input_vocab.update(item["tokenized_informal"])
        output_vocab.update(item["tokenized_formal"])

    input_vocab = {word: idx for idx, word in enumerate(input_vocab)}
    output_vocab = {word: idx for idx, word in enumerate(output_vocab)}

    return input_vocab, output_vocab


In [67]:
def pad_sequence(sequence, max_length, vocab):
    padded_sequence = [vocab[word] for word in sequence if word in vocab]
    padded_sequence += [0] * (max_length - len(padded_sequence))
    return padded_sequence

def numericalize_data(data, input_vocab, output_vocab):
#     max_input_length = max(len(item["tokenized_informal"]) for item in data)
#     max_output_length = max(len(item["tokenized_formal"]) for item in data)
    max_input_length = 65
    max_output_length = 65

    for item in data:
        item["input_seq"] = pad_sequence(item["tokenized_informal"], max_input_length, input_vocab)
        item["output_seq"] = pad_sequence(item["tokenized_formal"], max_output_length, output_vocab)

    return data


Defining a Bi-Directional RNN Model

In [68]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.birnn = nn.LSTM(hidden_size, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, len(output_vocab))

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.birnn(embedded)
        output = self.fc(output)
        return torch.nn.functional.log_softmax(output, dim=-1)


Training function

In [69]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            inputs = batch["input_seq"].to(device)
            targets = batch["output_seq"].to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs.view(-1, len(output_vocab)), targets.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")


preparing the data

In [70]:
data = preprocess_data(dataset)
data = tokenize_data(data)


In [71]:
print(data[1])

{'id': 1, 'topic': 'Family_Relationships', 'transformation': {'informal': 'Hmmm, I am a guy suffering from verbal abuse from my wife.', 'formal.ref0': 'I suffer through verbal abuse from my wife.', 'formal.ref1': '', 'formal.ref2': '', 'formal.ref3': ''}, 'tokenized_informal': ['Hmmm', ',', 'I', 'am', 'a', 'guy', 'suffering', 'from', 'verbal', 'abuse', 'from', 'my', 'wife', '.'], 'tokenized_formal': ['I', 'suffer', 'through', 'verbal', 'abuse', 'from', 'my', 'wife', '.']}


In [72]:
input_vocab, output_vocab = build_vocab(data)
data = numericalize_data(data, input_vocab, output_vocab)

In [73]:
print(len(data[0]['input_seq']))
print(len(data))

65
104562


In [74]:
train_data = data
train_dataset = CustomDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [75]:
model = BiRNN(len(input_vocab), 256, len(output_vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
trained_model = train_model(model, train_loader, criterion, optimizer)

Epoch 1/10: 100%|█████████████████████████████████████████████████████████| 3268/3268 [03:55<00:00, 13.89it/s]


Epoch 1, Loss: 5.649266742580232


Epoch 2/10: 100%|█████████████████████████████████████████████████████████| 3268/3268 [03:55<00:00, 13.86it/s]


Epoch 2, Loss: 5.179068140329901


Epoch 3/10: 100%|█████████████████████████████████████████████████████████| 3268/3268 [03:55<00:00, 13.86it/s]


Epoch 3, Loss: 4.962975626622157


Epoch 4/10:   7%|████▎                                                     | 240/3268 [00:17<03:38, 13.88it/s]

In [ ]:


# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# # Tokenize the informal sentences
# def preprocess_function(examples, input_field="informal", target_field="formal.ref0"):
#     inputs = [ex[input_field] for ex in examples["transformation"]]
#     targets = [ex[target_field] for ex in examples["transformation"]]

#     new_examples = tokenizer(
#         inputs, text_target=targets, max_length=64, truncation=True, padding="max_length"
#     )

#     return new_examples

# def create_multi_ref_dataset(dataset):
#   for i, target_field in enumerate(['formal.ref0', 'formal.ref1', 'formal.ref2', 'formal.ref3', 'rule_based_preprocessed']):
#     new_dataset = preprocess_function(dataset, 'informal', target_field)
#     dataset = dataset.add_column(f'labels_{i}', new_dataset['labels'])
#     if i == 0:
#       dataset = dataset.add_column('input_ids', new_dataset['input_ids'])
#       dataset = dataset.add_column('token_type_ids', new_dataset['token_type_ids'])
#       dataset = dataset.add_column('attention_mask', new_dataset['attention_mask'])

#   return dataset

# train_dataset = dataset['train'].map(
#     preprocess_function,
#     batched=True,
# )

# # tokenizer.convert_ids_to_tokens(train_dataset['input_ids'][0])


In [ ]:
print(train_dataset['input_ids'][0])